In [1]:
import pandas as pd
import bs4
import numpy as np

In [9]:
data_path = "../data/bridges.csv"
data_types = {'#': str,
              'Name': str,
              'Year': str,
              'Location': str,
              'Country': str,
              'Status': str
             }

bridges_df = pd.read_csv(data_path, dtype = data_types)

In [10]:
bridges_df

,#,Name,Year,Location,Country,Status
0,1,470 Odra River Bridge,2009,Ostrava,Czechia,in use
1,2,4708/C203 Bridge,2009,Ostrava,Czechia,in use
2,3,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use
3,4,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use
4,5,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use
...,...,...,...,...,...,...
15468,7947,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use
15469,7948,Zwischenbrunner Strasse Bridge,NaN,Sankt Pölten (NOE),Austria,in use
15470,7949,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use
15471,7950,Żywiec Bridge,NaN,Zywiec (SL),Poland,in use


In [14]:
bridges_df_no_na_year = bridges_df.dropna(subset = ['Year'])
bridges_df_no_na_year

,#,Name,Year,Location,Country,Status
0,1,470 Odra River Bridge,2009,Ostrava,Czechia,in use
1,2,4708/C203 Bridge,2009,Ostrava,Czechia,in use
2,3,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use
3,4,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use
4,5,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use
...,...,...,...,...,...,...
15465,7944,Zweigert Bridge,2000,Essen (NRW),Germany,in use
15467,7946,Zwierzyniecki Bridge,1897,Wroclaw (DS),Poland,in use
15468,7947,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use
15470,7949,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use


In [19]:
def classify_bridges(curr, prev, curr_category, next_category):
    if(curr != prev + 1):
        return next_category
    return curr_category        

In [17]:
bridges_df_no_na_year.apply(lambda x: classify_bridges(x.'#', ))
    
#1, 2, 3, 
#1, 2, 3, 4, 
#1, 2, 3, 
#1, 2
#bridges_df_no_na_year.drop(columns = ['#'])

,Name,Year,Location,Country,Status
0,470 Odra River Bridge,2009,Ostrava,Czechia,in use
1,4708/C203 Bridge,2009,Ostrava,Czechia,in use
2,7th Avenue Pedestrian Bridge (N1),2010,Johannesburg,South Africa,in use
3,A 1 Lenne Bridge,2009,Hagen (NRW),Germany,in use
4,A 113 Teltowkanal Bridge,2004,- Berlin (BE),Germany,in use
...,...,...,...,...,...
15465,Zweigert Bridge,2000,Essen (NRW),Germany,in use
15467,Zwierzyniecki Bridge,1897,Wroclaw (DS),Poland,in use
15468,Zwingenberg Bridge,2011,Zwingenberg (BW),Germany,in use
15470,Zwolle Railroad Bridge,2011,Zwolle,Netherlands,in use
